In [1]:
##
import numpy as np
import pandas as pd

import gc
import os
from matplotlib import pyplot as plt
import rasterio
import geopandas as gpd
from rasterio.plot import show, show_hist
import rasterio
import rasterio.mask as Mask
import rasterio.features
import rasterio.warp
from rasterio.plot import show_hist
import fiona
import os
import glob
import shapely
from shapely.geometry import LineString, Polygon, Polygon, Point, MultiPoint, MultiPolygon
from functools import partial
import pyproj
import shapely
from shapely.ops import transform
from tqdm import tqdm_notebook, tqdm
## help with figure rotation
from scipy import ndimage, misc
from shapely.geometry import Point
import geopandas as gpd
import cv2   # usse to resize images so that every image would have the same size!!!! 
## GDAL_DATA environment problem
os.environ['GDAL_DATA'] = '/opt/anaconda3/pkgs/libgdal-2.3.3-h0950a36_0/share/gdal'
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar


In [21]:
def concat_bands(name='L1C_T53HPA_A004418_20180110T005319', 
                 BASE = '/Volumes/My Passport for Mac/sentinel_2_53HPA/',
                 #OUT='/Volumes/My Passport for Mac/'
                 ):
    '''Resize all bands to (5490, 5490), resolution at 20meter/pixel'''

    b01 = rasterio.open(BASE+name+'_b01.tif').read(1)
    b01 = np.expand_dims(cv2.resize(b01, (5490,5490)), axis=2)[3000:4000,2200:3200,:]
    b09 = rasterio.open(BASE+name+'_b09.tif').read(1)
    b09 = np.expand_dims(cv2.resize(b09, (5490,5490)), axis=2)[3000:4000,2200:3200,:]
    b10 = rasterio.open(BASE+name+'_b10.tif').read(1)
    b10 = np.expand_dims(cv2.resize(b10, (5490,5490)), axis=2)[3000:4000,2200:3200,:]

    b02 = rasterio.open(BASE+name+'_b02.tif').read(1)
    b02 = np.expand_dims(cv2.resize(b02, (5490,5490)), axis=2)[3000:4000,2200:3200,:]
    b03 = rasterio.open(BASE+name+'_b03.tif').read(1)
    b03 = np.expand_dims(cv2.resize(b03, (5490,5490)), axis=2)[3000:4000,2200:3200,:]
    b04 = rasterio.open(BASE+name+'_b04.tif').read(1)
    b04 = np.expand_dims(cv2.resize(b04, (5490,5490)), axis=2)[3000:4000,2200:3200,:]
    b08 = rasterio.open(BASE+name+'_b08.tif').read(1)
    b08 = np.expand_dims(cv2.resize(b08, (5490,5490)), axis=2)[3000:4000,2200:3200,:]
    tci = rasterio.open(BASE+name+'_tci.tif').read(1)
    tci = np.expand_dims(cv2.resize(tci, (5490,5490)), axis=2)[3000:4000,2200:3200,:]

    b05 = np.expand_dims(rasterio.open(BASE+name+'_b05.tif').read(1), axis=2)[3000:4000,2200:3200,:]
    b06 = np.expand_dims(rasterio.open(BASE+name+'_b06.tif').read(1), axis=2)[3000:4000,2200:3200,:]
    b07 = np.expand_dims(rasterio.open(BASE+name+'_b07.tif').read(1), axis=2)[3000:4000,2200:3200,:]
    b8A = np.expand_dims(rasterio.open(BASE+name+'_b8A.tif').read(1), axis=2)[3000:4000,2200:3200,:]
    b11 = np.expand_dims(rasterio.open(BASE+name+'_b11.tif').read(1), axis=2)[3000:4000,2200:3200,:]
    b12 = np.expand_dims(rasterio.open(BASE+name+'_b12.tif').read(1), axis=2)[3000:4000,2200:3200,:]
    
    concat = np.concatenate([b01,b02,b03,b04,b05,b06,b07,b08,b09,b10,b11,b12,b8A,tci],axis=-1)
    concat = np.expand_dims(concat, 0)
    return concat

def expand_nbr(name='L1C_T53HPA_A004418_20180110T005319', 
                 BASE = '/Volumes/My Passport for Mac/sentinel_2_53HPA/',
                 #OUT='/Volumes/My Passport for Mac/'
                A=100
                 ):
    
    ## add 200 paddings outwards!
    smooth=1e-5
    b08 = rasterio.open(BASE+name+'_b08.tif').read(1)
    b08 = np.expand_dims(cv2.resize(b08, (5490,5490)), axis=2)[3000-A:4000+A,2200-A:3200+A,:]
    b12 = np.expand_dims(rasterio.open(BASE+name+'_b12.tif').read(1), axis=2)[3000-A:4000+A,2200-A:3200+A,:]
    nbr = (b08-b12+smooth)/(b08+b12+smooth)
    nbr = np.expand_dims(nbr, 0)
    return nbr


bigquery_res = pd.read_csv('/Volumes/My Passport for Mac/sentinel_2_53HPA/bigquery_results_s.csv', index_col=0)
bigquery_res['sensing_time'] = pd.to_datetime(bigquery_res['sensing_time'])
bigquery_res = bigquery_res[bigquery_res['cloud_cover']<75]
bigquery_res = bigquery_res.reset_index(drop=True)


In [30]:
id_names = bigquery_res['granule_id'].values.tolist()
id_names.extend(['L1C_T53HPA_A015000_20200120T005618','L1C_T53HPA_A023980_20200125T004659'])
id_names = id_names[-6:]


In [31]:
for i in tqdm_notebook(range(len(id_names))):
    if i == 0:
        imgs = concat_bands(id_names[i])
        exp_nbr = expand_nbr(name=id_names[i])
    else:
        imgs = np.concatenate([imgs, concat_bands(id_names[i])])
        exp_nbr = np.concatenate([exp_nbr, expand_nbr(name=id_names[i])],0)


In [32]:
exp_dnbr = np.zeros((5,1200,1200,1))
for i in range(5):
    exp_dnbr[i,:] = exp_nbr[i+1,:] - exp_nbr[i,:]

In [33]:
dnbr_s = exp_dnbr[:,200:1200,100:1100,:]
dnbr_n = exp_dnbr[:,0:1000,100:1100,:]
dnbr_w = exp_dnbr[:,100:1100,0:1000,:]
dnbr_e = exp_dnbr[:,100:1100,200:1200,:]

dnbr_nw = exp_dnbr[:,0:1000,0:1000,:]
dnbr_sw = exp_dnbr[:,200:1200,0:1000,:]
dnbr_ne = exp_dnbr[:,0:1000,200:1200,:]
dnbr_se = exp_dnbr[:,200:1200,200:1200,:]
dnbr_list  = [dnbr_s, dnbr_n, dnbr_w, dnbr_e, dnbr_nw, dnbr_sw, dnbr_ne, dnbr_se]
suffix_list= ['south','north','west','east','northwest','southwest','northeast','southeast']



In [34]:
smooth = 1e-5
b08 = imgs[:,:,:,7:8] 
b12 = imgs[:,:,:,11:12]
nbr = (b08-b12+smooth)/(b08+b12+smooth)

dnbr = np.zeros((5,1000,1000,1))
for i in range(5):
    dnbr[i,:] = nbr[i+1,:] - nbr[i,:]

In [35]:
dnbr.shape

(5, 1000, 1000, 1)

In [37]:
#np.savez_compressed('test_data', exp_dnbr=exp_dnbr, imgs=imgs, dnbr=dnbr)


In [ ]:
### Generate test set gsod features
local_records_day_0 = pd.read_csv('/Users/jincui/Downloads/landsat_analysis/fc_day_0.csv').drop(['date'],1)
local_records_day_1 = pd.read_csv('/Users/jincui/Downloads/landsat_analysis/fc_day_1.csv').drop(['date'],1)
local_records_day_2 = pd.read_csv('/Users/jincui/Downloads/landsat_analysis/fc_day_2.csv').drop(['date'],1)
local_records_day_3 = pd.read_csv('/Users/jincui/Downloads/landsat_analysis/fc_day_3.csv').drop(['date'],1)
local_records_day_4 = pd.read_csv('/Users/jincui/Downloads/landsat_analysis/fc_day_4.csv').drop(['date'],1)

local_records_day_0.columns = [*map(lambda t: t+'_0', local_records_day_0.columns)]
local_records_day_1.columns = [*map(lambda t: t+'_1', local_records_day_1.columns)]
local_records_day_2.columns = [*map(lambda t: t+'_2', local_records_day_2.columns)]
local_records_day_3.columns = [*map(lambda t: t+'_3', local_records_day_3.columns)]
local_records_day_4.columns = [*map(lambda t: t+'_4', local_records_day_4.columns)]
gsod_5day = pd.concat([local_records_day_0.iloc[-2:,:],
           local_records_day_1.iloc[-2:,:], 
           local_records_day_2.iloc[-2:,:], 
           local_records_day_3.iloc[-2:,:], 
           local_records_day_4.iloc[-2:,:]
          ],1)

# Had to manuall fill in some weather data from https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2020/
##gsod_5day.to_csv('test_gsod_features.csv', index=False)